In [7]:

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [8]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
wiki_soup = BeautifulSoup(wiki.content)


# <a rel="nofollow" class="external text" href="https://www.nyse.com/quote/XNYS:MMM">MMM</a>
symbols = (
    [x.get_text() 
           for x in 
           wiki_soup.find_all(attrs={'class':'external text'}) if x.get_text()!='reports'][:505]
)

In [9]:
symbols[:5]

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']

In [10]:
# def get_companies(symbols):
#     companies = []
#     for symbol in symbols:
#         try:
#             request = requests.get(f'https://datafied.api.edgar-online.com/v2/companies?primarysymbols={symbol}&appkey={config.edgar_apiKey}')
#             response = request.json()
#             companies.append({x['field']:x['value'] for x in response['result']['rows'][0]['values']})
#         except:
#             continue
#     return companies
def get_companies(symbols):
    companies = []

    request = requests.get(f'https://datafied.api.edgar-online.com/v2/companies?primarysymbols={symbols}&appkey={config.edgar_apiKey}')
    response = request.json()
    companies = ({x['field']:x['value'] for x in response['result']['rows'][0]['values']})

    return companies

def corefinancials(primary_symbol):
    request = requests.get(f"""https://datafied.api.edgar-online.com/v2/corefinancials/ann?primarysymbols={primary_symbol}&numperiods=1&appkey={config.edgar_apiKey}""")
    return request.json()

def get_coreFinancials(symbols):
    dfs = []
    for symbol in symbols:
        try:
            contents = corefinancials(symbol)
            financials = {x['field']:x['value'] for x in contents['result']['rows'][0]['values']}
            temp_df = pd.DataFrame(financials,index=[financials['cik']])
            temp_df['sic'] = get_sic(symbol)
            dfs.append(temp_df)
        except:
            continue
    return pd.concat(dfs,sort=True)

In [11]:
get_companies(symbols)

NameError: name 'config' is not defined

In [137]:
\financials.isna().sum(axis=0).sort_values(ascending=False)[:30]

extraordinaryitems                      453
deferredcharges                         435
otherequity                             409
discontinuedoperations                  383
researchdevelopmentexpense              349
equityearnings                          348
othercurrentliabilities                 241
othercurrentassets                      230
minorityinterest                        228
preferredstock                          225
changeininventories                     195
treasurystock                           189
changeinaccountsreceivable              167
inventoriesnet                          161
effectofexchangerateoncash              159
intangibleassets                        153
interestexpense                         108
totalshorttermdebt                      101
totalcurrentassets                       95
totalcurrentliabilities                  95
investmentchangesnet                     87
goodwill                                 86
costofrevenue                   

In [88]:
wiki_table = wiki_soup.find('table',{'id':'constituents'})

In [96]:
table = [x.get_text() for x in wiki_table.find_all(['td'])]

In [112]:
table

['MMM\n',
 '3M Company',
 'reports',
 'Industrials',
 'Industrial Conglomerates',
 'St. Paul, Minnesota',
 '',
 '0000066740',
 '1902\n',
 'ABT\n',
 'Abbott Laboratories',
 'reports',
 'Health Care',
 'Health Care Equipment',
 'North Chicago, Illinois',
 '1964-03-31',
 '0000001800',
 '1888\n',
 'ABBV\n',
 'AbbVie Inc.',
 'reports',
 'Health Care',
 'Pharmaceuticals',
 'North Chicago, Illinois',
 '2012-12-31',
 '0001551152',
 '2013 (1888)\n',
 'ABMD\n',
 'ABIOMED Inc',
 'reports',
 'Health Care',
 'Health Care Equipment',
 'Danvers, Massachusetts',
 '2018-05-31',
 '0000815094',
 '1981\n',
 'ACN\n',
 'Accenture plc',
 'reports',
 'Information Technology',
 'IT Consulting & Other Services',
 'Dublin, Ireland',
 '2011-07-06',
 '0001467373',
 '1989\n',
 'ATVI\n',
 'Activision Blizzard',
 'reports',
 'Communication Services',
 'Interactive Home Entertainment',
 'Santa Monica, California',
 '2015-08-31',
 '0000718877',
 '2008\n',
 'ADBE\n',
 'Adobe Systems Inc',
 'reports',
 'Information Techn

In [122]:
tuple(zip(table[0::9],table[3::9]))

(('MMM\n', 'Industrials'),
 ('ABT\n', 'Health Care'),
 ('ABBV\n', 'Health Care'),
 ('ABMD\n', 'Health Care'),
 ('ACN\n', 'Information Technology'),
 ('ATVI\n', 'Communication Services'),
 ('ADBE\n', 'Information Technology'),
 ('AMD\n', 'Information Technology'),
 ('AAP\n', 'Consumer Discretionary'),
 ('AES\n', 'Utilities'),
 ('AMG\n', 'Financials'),
 ('AFL\n', 'Financials'),
 ('A\n', 'Health Care'),
 ('APD\n', 'Materials'),
 ('AKAM\n', 'Information Technology'),
 ('ALK\n', 'Industrials'),
 ('ALB\n', 'Materials'),
 ('ARE\n', 'Real Estate'),
 ('ALXN\n', 'Health Care'),
 ('ALGN\n', 'Health Care'),
 ('ALLE\n', 'Industrials'),
 ('AGN\n', 'Health Care'),
 ('ADS\n', 'Information Technology'),
 ('LNT\n', 'Utilities'),
 ('ALL\n', 'Financials'),
 ('GOOGL\n', 'Communication Services'),
 ('GOOG\n', 'Communication Services'),
 ('MO\n', 'Consumer Staples'),
 ('AMZN\n', 'Consumer Discretionary'),
 ('AMCR\n', 'Materials'),
 ('AEE\n', 'Utilities'),
 ('AAL\n', 'Industrials'),
 ('AEP\n', 'Utilities'),
 

In [120]:
[table[x::9] for x in range(0,9)]


[['MMM\n',
  'ABT\n',
  'ABBV\n',
  'ABMD\n',
  'ACN\n',
  'ATVI\n',
  'ADBE\n',
  'AMD\n',
  'AAP\n',
  'AES\n',
  'AMG\n',
  'AFL\n',
  'A\n',
  'APD\n',
  'AKAM\n',
  'ALK\n',
  'ALB\n',
  'ARE\n',
  'ALXN\n',
  'ALGN\n',
  'ALLE\n',
  'AGN\n',
  'ADS\n',
  'LNT\n',
  'ALL\n',
  'GOOGL\n',
  'GOOG\n',
  'MO\n',
  'AMZN\n',
  'AMCR\n',
  'AEE\n',
  'AAL\n',
  'AEP\n',
  'AXP\n',
  'AIG\n',
  'AMT\n',
  'AWK\n',
  'AMP\n',
  'ABC\n',
  'AME\n',
  'AMGN\n',
  'APH\n',
  'ADI\n',
  'ANSS\n',
  'ANTM\n',
  'AON\n',
  'AOS\n',
  'APA\n',
  'AIV\n',
  'AAPL\n',
  'AMAT\n',
  'APTV\n',
  'ADM\n',
  'ARNC\n',
  'ANET\n',
  'AJG\n',
  'AIZ\n',
  'ATO\n',
  'T\n',
  'ADSK\n',
  'ADP\n',
  'AZO\n',
  'AVB\n',
  'AVY\n',
  'BKR\n',
  'BLL\n',
  'BAC\n',
  'BK\n',
  'BAX\n',
  'BBT\n',
  'BDX\n',
  'BRK.B\n',
  'BBY\n',
  'BIIB\n',
  'BLK\n',
  'BA\n',
  'BKNG\n',
  'BWA\n',
  'BXP\n',
  'BSX\n',
  'BMY\n',
  'AVGO\n',
  'BR\n',
  'BF.B\n',
  'CHRW\n',
  'COG\n',
  'CDNS\n',
  'CPB\n',
  'COF\n',

In [141]:
financials['cik']

MMM     0000066740
ABT     0000001800
ABBV    0001551152
ABMD    0000815094
ACN     0001467373
           ...    
XYL     0001524472
YUM     0001041061
ZBH     0001136869
ZION    0000109380
ZTS     0001555280
Name: cik, Length: 497, dtype: object